Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [25]:
NAME = "Stephanya CASANOVA MARROQUIN"
COLLABORATORS = ""

---

# Machine Learning in Spark

Following the evolution of Spark, there are two ways to do Machine Learning on Spark :

* MLlib, or `spark.mllib`, was the first ML library implemented in the core Spark library and runs on RDDs. As of today, the library is in maintenance mode, but as we did for RDDs vs DataFrames, it is important that we cover some aspects of the older library. MLlib is also the only library that supports training models for Spark Streaming. 
* ML, or `spark.ml` is now the primary ML library on Spark, and runs on DataFrames. Its API is close to those of other mainstream librairies like scikit-learn.

We will dive into both APIs in this notebook, using the `titanic.csv` file for classification purposes on the `Survived` column.

_I think at this point of your career, you all know what the [Titanic dataset](https://www.kaggle.com/c/titanic/data) is..._

In [26]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('lecture-lyon2').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [27]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.mllib.linalg import VectorUDT

---
## Data preparation

Even though MLlib is designed with RDDs and DStreams in focus, for ease of transforming the data we will read the data and convert it to a DataFrame. Afterwards we will build RDDs for training in MLlib, or stay in DataFrame for training in ML.

In [28]:
filePath = 'titanic.csv'
data = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(filePath)
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [29]:
data.describe().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


From the first summary statistics, we see that the `Age`, `Cabin` and `Embarked` variables can have null values. Also `PassengerId` and `Ticket` look useless for future predictions.

# Question
  
* Drop `Cabin`, `Ticket` and `PassengerId`
* Using `.na.fill` function on a DataFrame :
    * For `Age`, replace `None` by the mean value for the column. 
    * For `Embarked` columns, replace `None` by the most frequent value for the column. 

In [30]:
#data=data.drop('PassengerId','Ticket','Cabin')
#mean = data.agg({"Age":"avg"}).collect()[0]["avg(Age)"]
#freq = "S"
#data=data.na.fill(freq,subset=["Embarked"])
#data=data.na.fill(mean,subset=["Age"])
#data.show()
#data.describe().toPandas()

In [31]:
def replace_na(df):
    # YOUR CODE HERE
    data = df
    data=data.drop('PassengerId','Ticket','Cabin')
    mean = data.agg({"Age":"avg"}).collect()[0]["avg(Age)"]
    freq = "S"
    data=data.na.fill(freq,subset=["Embarked"])
    data=data.na.fill(mean,subset=["Age"])
    return data

In [32]:
"""
Graded cell

3 points
"""
result = replace_na(data)
assert float(result.describe().toPandas().loc[2]['Age']) - 13 < 0.1
assert int(result.describe().toPandas().loc[0]['Embarked']) == 891
assert list(result.toPandas().columns.values) == ['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

For the following two questions, we will use [Transformers](https://spark.apache.org/docs/2.2.0/ml-pipeline.html#transformers). Technically, a Transformer implements a method `transform()`, which converts one DataFrame into another, generally by appending one or more columns.

Example: 

```python
from pyspark.ml.feature import Binarizer

continuousDataFrame = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2)
], ["id", "feature"])
continuousDataFrame.show()

binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")
binarizedDataFrame = binarizer.transform(continuousDataFrame)
print("Binarizer output with Threshold = %f" % binarizer.getThreshold())
binarizedDataFrame.show()
```

Result :

```
+---+-------+
| id|feature|
+---+-------+
|  0|    0.1|
|  1|    0.8|
|  2|    0.2|
+---+-------+

Binarizer output with Threshold = 0.500000
+---+-------+-----------------+
| id|feature|binarized_feature|
+---+-------+-----------------+
|  0|    0.1|              0.0|
|  1|    0.8|              1.0|
|  2|    0.2|              0.0|
+---+-------+-----------------+
```

**Note:** contrary to previous notebooks, I have not imported all of the libraries needed to solve the remaining exercises. When you want to import a library, please import it in the same notebook cell as where you implement your code, otherwise it may impact the automatic grading.

# Question

Through some regex, the [regex_extract UDF](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+UDF) and [SQLTransformer](https://spark.apache.org/docs/2.2.0/ml-features.html#sqltransformer), get the title of a person from the `Name` column in a `Title` column. Drop the `Name` column afterwards.

Example

```
Braund, Mr. Owen       --> Mr
Andria, Doctor. Steve  --> Doctor
```

_Not a hint: while it is perfectly possible to write a custom UDF to solve this question, it breaks the purpose of using Dataframes for cleaning because UDFs don't benefit from SparkSQL's optimizer engine and have to transform back to Java objects for processing. Spark built-in UDFs don't share this problem._

In [33]:
#data.toPandas()["Name"]
#type()
#from pyspark.sql.functions import regexp_extract, col
#result = data.withColumn('Civility', regexp_extract(col('Name'), ', (.+?)\. ',1))
#list(result.select('Civility').distinct().toPandas()['Civility'].sort_values().values)

In [34]:
def extract_civility(df):    
    """
    Return dataframe dropping Name and replacing with Title
    """
    # YOUR CODE HERE
    from pyspark.sql.functions import regexp_extract, col
    result = data.withColumn('Civility', regexp_extract(col('Name'), ', (.+?)\. ',1))
    result = result.drop("Name")
    return result

In [35]:
"""
Graded cell

4 points
"""
result = extract_civility(data)
resultCols = result.columns
assert 'Name' not in resultCols
assert 'Civility' in resultCols
assert list(result.select('Civility').distinct().toPandas()['Civility'].sort_values().values) == [
    'Capt',
    'Col',
    'Don',
    'Dr',
    'Jonkheer',
    'Lady',
    'Major',
    'Master',
    'Miss',
    'Mlle',
    'Mme',
    'Mr',
    'Mrs',
    'Ms',
    'Rev',
    'Sir',
    'the Countess'
]

# Question 

[One hot encode](https://spark.apache.org/docs/2.2.0/ml-features.html#onehotencoder) `Sex`, `Civility` and `Embarked` columns into `SexVec`, `CivilityVec` and `EmbarkedVec`. Don't forget to drop the original columns.

In [36]:
def one_hot_encode(df):
    """
    Return dataframe one hot encoding selected columns    
    """
    from pyspark.ml.feature import OneHotEncoder, StringIndexer
    
    stringSexIndexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
    modelSex = stringSexIndexer.setHandleInvalid("skip").fit(df)
    indexedSex = modelSex.transform(df)
    encoderSex = OneHotEncoder(inputCol="SexIndex", outputCol="SexVec")
    df = encoderSex.transform(indexedSex)

    stringCivilityIndexer = StringIndexer(inputCol="Civility", outputCol="CivilityIndex")
    modelCivility = stringCivilityIndexer.setHandleInvalid("skip").fit(df)
    indexedCivility = modelCivility.transform(df)
    encoderCivility = OneHotEncoder(inputCol="CivilityIndex", outputCol="CivilityVec")
    df = encoderCivility.transform(indexedCivility)

    stringEmbarkedIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
    modelEmbarked = stringEmbarkedIndexer.setHandleInvalid("skip").fit(df)
    indexedEmbarked = modelEmbarked.transform(df)
    encoderEmbarked = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
    df = encoderEmbarked.transform(indexedEmbarked)

    df = df.drop('Sex','SexIndex','Civility','CivilityIndex','Embarked','EmbarkedIndex')

    return df

In [37]:
"""
Graded cell

4 points
"""
result = one_hot_encode(extract_civility(data))
resultCols = result.columns
assert len(resultCols) == 12

assert 'SexVec' in resultCols
assert 'CivilityVec' in resultCols
assert 'EmbarkedVec' in resultCols

assert 'Sex' not in resultCols
assert 'Civility' not in resultCols
assert 'Embarked' not in resultCols

assert result.schema['SexVec'].simpleString() == 'SexVec:vector'
assert result.schema['CivilityVec'].simpleString() == 'CivilityVec:vector'
assert result.schema['EmbarkedVec'].simpleString() == 'EmbarkedVec:vector'

# Question

Now that we have created all of our numeric features, we need to assemble them into the same column. This is the goal of the [VectorAssembler](https://spark.apache.org/docs/2.2.0/ml-features.html#vectorassembler) transformer.

In [38]:
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler
#df = data
#assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch'], outputCol="features")
#result = assembler.setHandleInvalid("skip").transform(df)
#result.select('features').show()

In [39]:
def feature_assemble(df, featureCols):
    """
    Assemble all features in the featureCols list into one column called 'features'.
    """
    # YOUR CODE HERE
    from pyspark.ml.linalg import Vectors
    from pyspark.ml.feature import VectorAssembler

    assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
    result = assembler.setHandleInvalid("skip").transform(df)
    
    return result

In [40]:
"""
Graded cell

2 points
"""
result = feature_assemble(data, ['Pclass', 'SibSp', 'Parch'])
assert 'features' in result.columns
assert result.schema['features'].simpleString() == 'features:vector'

---
#### All the data preparation has been made. After running the following cell, we can concentrate on running ML modelling.

For comparison purposes, let's try a Logistic Regression from MLlib and ML on the dataset.

In [41]:
# prepare the data !
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'SexVec', 'CivilityVec', 'EmbarkedVec']
prepared_data = feature_assemble(one_hot_encode(extract_civility(replace_na(data))), features)
prepared_data = prepared_data.withColumnRenamed("Survived", "label").select(['label', 'features'])
train, test = prepared_data.randomSplit([0.75, 0.25], 0)

train.cache()
test.cache()

DataFrame[label: int, features: vector]

---
## MLlib - RDD based API

We will first use the RDD-based [Logistic Regression](https://spark.apache.org/docs/2.2.0/mllib-linear-methods.html#logistic-regression). The exercise comes into two steps :

1. First, you must create a RDD of LabeledPoint(label, features). Also careful as we are using `pyspark.ml.linalg.SparseVector` but the RDD-based API expects `pyspark.mllib.linalg.SparseVector`, so we need to [convert it](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html?highlight=linearregressionwithsgd#pyspark.mllib.linalg.Vectors.fromML).
2. Then you can apply LogisticRegression on it.

# Question

Train a logistic regression model on the train dataset.

In [42]:
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.linalg import Vectors
#from pyspark.mllib.evaluation import BinaryClassificationMetrics
#from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
#train_r= train.rdd.map(lambda row: LabeledPoint(row[0], Vectors.fromML(row[1])))
#model = LogisticRegressionWithLBFGS.train(train_r, iterations=10, numClasses=2)
#predictionAndLabels = test_rdd.map(lambda lp: (float(model.predict(lp.features)), lp.label))
#metrics = BinaryClassificationMetrics(predictionAndLabels)
#metrics.areaUnderROC

In [43]:
def dataframe_to_labeledpoints(df):
    """
    This function takes the conversion from a DataFrame of columns [label, features] to a RDD of LabeledPoint.    
    """
    from pyspark.mllib.regression import LabeledPoint
    from pyspark.mllib.linalg import Vectors
    
    return df.rdd.map(lambda row: LabeledPoint(row[0], Vectors.fromML(row[1])))

def train_mllib_logistic(train):
    """
    Return a MLlib logistic regression trained on a RDD of LabeledPoint. 
    """
    # YOUR CODE HERE
    from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
    model = LogisticRegressionWithLBFGS.train(train, iterations=10, numClasses=2)
    return model

In [44]:
"""
Graded cell

4 points
"""
from pyspark.mllib.evaluation import BinaryClassificationMetrics

train_rdd = dataframe_to_labeledpoints(train)
test_rdd = dataframe_to_labeledpoints(test)
model = train_mllib_logistic(train_rdd)

predictionAndLabels = test_rdd.map(lambda lp: (float(model.predict(lp.features)), lp.label))

metrics = BinaryClassificationMetrics(predictionAndLabels)
assert metrics.areaUnderROC > 0.75  # I managed ~0.8 on my first try

---
## ML - DataFrame based API

We now compare with using the ML [Logistic regression](https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#binomial-logistic-regression). It should work directly on our dataset.

In [45]:
from pyspark.ml.classification import LogisticRegression
lr=LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
model = lr.fit(train)
model.summary.areaUnderROC

0.7922032693674484

In [46]:
def train_ml_logistic(train):
    """
    Return a MLlib logistic regression trained on a RDD of LabeledPoint. 
    """
    # YOUR CODE HERE
    from pyspark.ml.classification import LogisticRegression
    lr=LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
    model = lr.fit(train)
    return model

In [47]:
"""
Graded cell

3 points
"""
from pyspark.ml.evaluation import BinaryClassificationEvaluator

model = train_ml_logistic(train)
assert model.summary.areaUnderROC > 0.75 # managed 0.87 on my first try

predictions = model.transform(test)
evaluator = BinaryClassificationEvaluator()
assert evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"}) > 0.75 # managed 0.88 on my first try

In [24]:
spark.stop()